In [2]:
!pip3 install praw

     |████████████████████████████████| 152 kB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 200 kB 9.9 MB/s eta 0:00:01


In [3]:
%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
import praw

reddit = praw.Reddit(
    client_id="8MguWY6Vv2iwDw",  
    client_secret="GyXPBkZaNZn1GT8o_K3lNXqx8mU",
    user_agent="Chrome:EloR Miner:v0.1 (by u/inutard)")

# From a glance at the daily top threads, gaming seems to be most popular
# We'll mine from a specific subreddit to make sure theres some overlap in the user base
thread_scores = []
subreddit_name = "SubredditSimulator"
# Each thread of O(1000) comments takes about 3min to process
count = 0
for submission in reddit.subreddit("SubredditSimulator").top(limit=2000):
    if count % 100 == 0:
        print(submission.title, submission.num_comments, submission.score)
    count += 1
    submission.comments.replace_more(limit=None)
    scores = []
    seen = set()
    for comment in submission.comments:
        try:
            scores.append((comment.score, comment.author.name))
            seen.add(comment.author.name)
        except:
            # Likely deleted comments
            pass
        
        # Include second-level and third-level comments too
        for reply in comment.replies:
            try:
                if reply.author.name not in seen:
                    scores.append((reply.score, reply.author.name))
                    seen.add(reply.author.name)
            except:
                # Likely deleted comments
                pass
    thread_scores.append(scores)

<--------------Number of people that think Donald Trump has dismissed FBI Director James Comey 66 64223


In [ ]:
thread_scores[0]

In [ ]:
# Total number of unique commenters
len(set(s[1] for score in thread_scores for s in score))

In [ ]:
# Frequencies of top posters
from collections import Counter
C = Counter(list(s[1] for score in thread_scores for s in score))
C.most_common(50)

In [ ]:
# Total number of comments
sum(len(s) for s in thread_scores)

In [ ]:
thread_scores[0]

In [ ]:
import json
from collections import defaultdict

for tid, thread in enumerate(thread_scores):
    data = {}
    data['id'] = tid
    data['name'] = "To be filled."
    data['time_seconds'] = 0
    
    standings = []
    data['standings'] = standings
    
    names = set()
    # Remove duplicate comments by consolidating votes
    together = defaultdict(int)
    for user in thread:
        together[user[1]] += user[0]
    thread = [(x[1], x[0]) for x in together.items()]
    
    thread = sorted(thread, reverse=True)
    # Use -1 as placeholder value
    lscore, lo, hi = -1, -1, -1
    backlog = []
    for user in thread:
        if user[0] != lscore:
            for name in backlog:
                standings.append([name, lo, hi])
            lo = hi = hi + 1
            backlog = []
        else:
            hi += 1
        backlog.append(user[1])
        lscore = user[0]
        
    for name in backlog:
        standings.append([name, lo, hi])
    
    with open('../cache/reddit/' + str(tid) + '.json', 'w') as out:
        json.dump(data, out)

In [ ]:
contest_ids = list(range(len(thread_scores)))
with open('../data/reddit/contest_ids.json', 'w') as out:
    out.write(str(contest_ids))

In [ ]:
usernames = set(data[1] for thread in thread_scores for data in thread)

In [ ]:
karmalist = []
for name in usernames:
    user = reddit.redditor(name)
    karmalist.append((user.comment_karma, user.name))
    
karmalist = sorted(karmalist, reverse=True)

In [ ]:
with open('../data/reddit/true_rankings.json', 'w') as out:
    out.write(str(karmalist))

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

with open('../data/reddit/predicted_rankings.txt') as pfile:
    plines = pfile.readlines()[12:]
    joined_data = defaultdict(list)
    for data in karmalist:
        joined_data[data[1]].append(data[0])
    for line in plines:
        tokens = line.split()
        rating = int(tokens[1].split('(')[0])
        name = tokens[2]
        joined_data[name].append(rating)
        
    # Basically we can delete any users that do not end in SS
    for user in usernames:
        if 'SS' != user[-2:]:
            del joined_data[user]
        
    x, y = zip(*list(joined_data.values()))
    plt.scatter(x, y, s=10)